# Title: {Title Here}

|         |            |
|---|:-------------|
| Version     | 0.1 |
| Author      | {author}      |
| Contact | {email}      |
| Data tables used | {tablelist here...                                                      }|

## Description:
{the description}


<a id='toc'></a>
## Contents
- [Setup and Authenticate](#setup)

- [First Section](#sectionone)
- [More Sections](#moresections)
- [Appendices](#appendices)
  - [Saving data to Excel](#appendices)


<a id='setup'></a>[Contents](#toc)
# Setup

If you are running this from an Azure Notebooks instance created by ASI you can ignore the *Install Packages* step.


## Install Packages

In [ ]:
# You may needs these - should only need to uncomment and run once
# !pip install msgpack
# !pip install Kqlmagic --no-cache-dir  --upgrade

# !pip install PyHamcrest
# !conda install -c conda-forge python-levenshtein -y
# !conda install requests
# !conda install attrs
# !conda install seaborn
# !conda install bokeh
# !conda install holoviews

# our package
#!pip install ../python --upgrade


### Import Python Packages

In [2]:
# Imports
import sys
MIN_REQ_PYTHON = (3,6)
if sys.version_info < MIN_REQ_PYTHON:
    print('Check the Kernel->Change Kernel menu and ensure that Python 3.6')
    print('or later is selected as the active kernel.')
    sys.exit("Python %s.%s or later is required.\n" % MIN_REQ_PYTHON)


import numpy as np
from IPython import get_ipython
from IPython.display import display, HTML
import ipywidgets as widgets

import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
sns.set()
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)

import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
   
import msticpy.sectools as sectools
import msticpy.asitools as asi
import msticpy.asitools.kql as qry
import msticpy.asitools.nbdisplay as asidisp


### Enter or confirm WorkspaceId
To find your Workspace Id go to [Log Analytics](#https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.OperationalInsights%2Fworkspaces). Look at the workspace properties to find the ID.

In [23]:
import os
from msticpy.asitools.asiconfig import WorkspaceConfig
ws_config_file = 'config.json'
try:
    ws_config = WorkspaceConfig(ws_config_file)
    print('Found config file')
    for cf_item in ['tenant_id', 'subscription_id', 'resource_group', 'workspace_id', 'workspace_name']:
        print(cf_item, ws_config[cf_item])
except:
    ws_config = None

ws_id = asi.GetEnvironmentKey(env_var='WORKSPACE_ID',
                              prompt='Log Analytics Workspace Id:')
if ws_config:
    ws_id.value = ws_config['workspace_id']
ws_id.display()

### Authenticate to Log Analytics
If you are using user/device authentication, run the following cell. 
- Click the 'Copy code to clipboard and authenticate' button.
- This will pop up an Azure Active Directory authentication dialog (in a new tab or browser window). The device code will have been copied to the clipboard. 
- Select the text box and paste (Ctrl-V/Cmd-V) the copied value. 
- You should then be redirected to a user authentication page where you should authenticate with a user account that has permission to query your Log Analytics workspace.

Use the following syntax if you are authenticating using an Azure Active Directory AppId and Secret:
```
%kql loganalytics://tenant(aad_tenant).workspace(WORKSPACE_ID).clientid(client_id).clientsecret(client_secret)
```
instead of
```
%kql loganalytics://code().workspace(WORKSPACE_ID)
```

In [10]:
try:
    WORKSPACE_ID = select_ws.value
except NameError:
    try:
        WORKSPACE_ID = ws_id.value
    except NameError:
        WORKSPACE_ID = None
    
if not WORKSPACE_ID:
    raise ValueError('No workspace selected.')

asi.kql.load_kql_magic()

%kql loganalytics://code().workspace(WORKSPACE_ID)

<a id='sectionone'></a>[Contents](#toc)
# Section One

Specify a time range to search for X.

In [11]:
query_times = asi.QueryTime(units='day', max_before=20, before=5, max_after=1)
query_times.display()

HTML(value='<h4>Set query time boundaries</h4>')

Type a query

In [21]:
start = f'datetime({query_times.start})'
end = f'datetime({query_times.end})'
start

'datetime(2019-01-31 00:52:43.268207)'

In [22]:
%kql SecurityEvent | where TimeGenerated >={start} and TimeGenerated <={end} | take 1


TenantId,TimeGenerated,SourceSystem,Account,AccountType,Computer,EventSourceName,Channel,Task,Level,EventData,EventID,Activity,PartitionKey,RowKey,StorageAccount,AzureDeploymentID,AzureTableName,AccessList,AccessMask,AccessReason,AccountDomain,AccountExpires,AccountName,AccountSessionIdentifier,AdditionalInfo,AdditionalInfo2,AllowedToDelegateTo,Attributes,AuditPolicyChanges,AuditsDiscarded,AuthenticationLevel,AuthenticationPackageName,AuthenticationProvider,AuthenticationServer,AuthenticationService,AuthenticationType,CACertificateHash,CalledStationID,CallerProcessId,CallerProcessName,CallingStationID,CAPublicKeyHash,CategoryId,CertificateDatabaseHash,ClassId,ClassName,ClientAddress,ClientIPAddress,ClientName,CommandLine,CompatibleIds,DCDNSName,DeviceDescription,DeviceId,DisplayName,Disposition,DomainBehaviorVersion,DomainName,DomainPolicyChanged,DomainSid,EAPType,ElevatedToken,ErrorCode,ExtendedQuarantineState,FailureReason,FileHash,FilePath,FilePathNoUser,Filter,ForceLogoff,Fqbn,FullyQualifiedSubjectMachineName,FullyQualifiedSubjectUserName,GroupMembership,HandleId,HardwareIds,HomeDirectory,HomePath,ImpersonationLevel,InterfaceUuid,IpAddress,IpPort,KeyLength,LmPackageName,LocationInformation,LockoutDuration,LockoutObservationWindow,LockoutThreshold,LoggingResult,LogonGuid,LogonHours,LogonID,LogonProcessName,LogonType,LogonTypeName,MachineAccountQuota,MachineInventory,MachineLogon,MandatoryLabel,MaxPasswordAge,MemberName,MemberSid,MinPasswordAge,MinPasswordLength,MixedDomainMode,NASIdentifier,NASIPv4Address,NASIPv6Address,NASPort,NASPortType,NetworkPolicyName,NewDate,NewMaxUsers,NewProcessId,NewProcessName,NewRemark,NewShareFlags,NewTime,NewUacValue,NewValue,NewValueType,ObjectName,ObjectServer,ObjectType,ObjectValueName,OemInformation,OldMaxUsers,OldRemark,OldShareFlags,OldUacValue,OldValue,OldValueType,OperationType,PackageName,ParentProcessName,PasswordHistoryLength,PasswordLastSet,PasswordProperties,PreviousDate,PreviousTime,PrimaryGroupId,PrivateKeyUsageCount,PrivilegeList,Process,ProcessId,ProcessName,Properties,ProfilePath,ProtocolSequence,ProxyPolicyName,QuarantineHelpURL,QuarantineSessionID,QuarantineSessionIdentifier,QuarantineState,QuarantineSystemHealthResult,RelativeTargetName,RemoteIpAddress,RemotePort,Requester,RequestId,RestrictedAdminMode,RowsDeleted,SamAccountName,ScriptPath,SecurityDescriptor,ServiceAccount,ServiceFileName,ServiceName,ServiceStartType,ServiceType,SessionName,ShareLocalPath,ShareName,SidHistory,Status,SubjectAccount,SubcategoryGuid,SubcategoryId,Subject,SubjectDomainName,SubjectKeyIdentifier,SubjectLogonId,SubjectMachineName,SubjectMachineSID,SubjectUserName,SubjectUserSid,SubStatus,TableId,TargetAccount,TargetDomainName,TargetInfo,TargetLinkedLogonId,TargetLogonGuid,TargetLogonId,TargetOutboundDomainName,TargetOutboundUserName,TargetServerName,TargetSid,TargetUser,TargetUserName,TargetUserSid,TemplateContent,TemplateDSObjectFQDN,TemplateInternalName,TemplateOID,TemplateSchemaVersion,TemplateVersion,TokenElevationType,TransmittedServices,UserAccountControl,UserParameters,UserPrincipalName,UserWorkstations,VirtualAccount,VendorIds,Workstation,WorkstationName,SourceComputerId,EventOriginId,MG,TimeCollected,ManagementGroupName,DestAddress_CF,DstAddress_CF,Type,_ResourceId
802d39e1-9d70-404d-832c-2de5e2478eda,2019-02-05 00:40:26.017000+00:00,OpsManager,WORKGROUP\v-arrikl-03020$,Machine,v-arrikl-03020,Microsoft-Windows-Security-Auditing,Security,13312,8,,4688,4688 - A new process has been created.,,,,,,,,,,,,,,,,,,None,None,,,,None,,,,,,,,,,,,,,,"""C:\windows\system32\cscript.exe"" /nologo ""MonitorKnowledgeDiscovery.vbs""",,,,,,,,,,,,,None,,,,,,,,,,,,,,,,,,,,None,,,,,,,,,,,None,,,,,S-1-16-16384,,,,,,,,,,,,,,,0x5a4,C:\Windows\System32\cscript.exe,,,,,,,,,,,,,,,,,,,,C:\Program Files\Microsoft Monitoring Agent\Agent\MonitoringHost.exe,,,,,,,,,cscript.exe,0x12ec,,,,,,,,,,,,,,,,,,,,,,,,None,,,,,,,WORKGROUP\v-arrikl-03020$,,,,WORKGROUP,,0x3e7,,,v-arrikl-03020$,S-1-5-18,,,-\-,-,,,,0x0,,,,,,-,S-1-0-0,,,

[{'TenantId': '802d39e1-9d70-404d-832c-2de5e2478eda', 'TimeGenerated': datetime.datetime(2019, 2, 5, 0, 40, 26, 17000, tzinfo=tzutc()), 'SourceSystem': 'OpsManager', 'Account': 'WORKGROUP\\v-arrikl-03020$', 'AccountType': 'Machine', 'Computer': 'v-arrikl-03020', 'EventSourceName': 'Microsoft-Windows-Security-Auditing', 'Channel': 'Security', 'Task': 13312, 'Level': '8', 'EventData': '', 'EventID': 4688, 'Activity': '4688 - A new process has been created.', 'PartitionKey': '', 'RowKey': '', 'StorageAccount': '', 'AzureDeploymentID': '', 'AzureTableName': '', 'AccessList': '', 'AccessMask': '', 'AccessReason': '', 'AccountDomain': '', 'AccountExpires': '', 'AccountName': '', 'AccountSessionIdentifier': '', 'AdditionalInfo': '', 'AdditionalInfo2': '', 'AllowedToDelegateTo': '', 'Attributes': '', 'AuditPolicyChanges': '', 'AuditsDiscarded': None, 'AuthenticationLevel': None, 'AuthenticationPackageName': '', 'AuthenticationProvider': '', 'AuthenticationServer': '', 'AuthenticationService': None, 'AuthenticationType': '', 'CACertificateHash': '', 'CalledStationID': '', 'CallerProcessId': '', 'CallerProcessName': '', 'CallingStationID': '', 'CAPublicKeyHash': '', 'CategoryId': '', 'CertificateDatabaseHash': '', 'ClassId': '', 'ClassName': '', 'ClientAddress': '', 'ClientIPAddress': '', 'ClientName': '', 'CommandLine': '"C:\\windows\\system32\\cscript.exe" /nologo "MonitorKnowledgeDiscovery.vbs"', 'CompatibleIds': '', 'DCDNSName': '', 'DeviceDescription': '', 'DeviceId': '', 'DisplayName': '', 'Disposition': '', 'DomainBehaviorVersion': '', 'DomainName': '', 'DomainPolicyChanged': '', 'DomainSid': '', 'EAPType': '', 'ElevatedToken': '', 'ErrorCode': None, 'ExtendedQuarantineState': '', 'FailureReason': '', 'FileHash': '', 'FilePath': '', 'FilePathNoUser': '', 'Filter': '', 'ForceLogoff': '', 'Fqbn': '', 'FullyQualifiedSubjectMachineName': '', 'FullyQualifiedSubjectUserName': '', 'GroupMembership': '', 'HandleId': '', 'HardwareIds': '', 'HomeDirectory': '', 'HomePath': '', 'ImpersonationLevel': '', 'InterfaceUuid': '', 'IpAddress': '', 'IpPort': '', 'KeyLength': None, 'LmPackageName': '', 'LocationInformation': '', 'LockoutDuration': '', 'LockoutObservationWindow': '', 'LockoutThreshold': '', 'LoggingResult': '', 'LogonGuid': '', 'LogonHours': '', 'LogonID': '', 'LogonProcessName': '', 'LogonType': None, 'LogonTypeName': '', 'MachineAccountQuota': '', 'MachineInventory': '', 'MachineLogon': '', 'MandatoryLabel': 'S-1-16-16384', 'MaxPasswordAge': '', 'MemberName': '', 'MemberSid': '', 'MinPasswordAge': '', 'MinPasswordLength': '', 'MixedDomainMode': '', 'NASIdentifier': '', 'NASIPv4Address': '', 'NASIPv6Address': '', 'NASPort': '', 'NASPortType': '', 'NetworkPolicyName': '', 'NewDate': '', 'NewMaxUsers': '', 'NewProcessId': '0x5a4', 'NewProcessName': 'C:\\Windows\\System32\\cscript.exe', 'NewRemark': '', 'NewShareFlags': '', 'NewTime': '', 'NewUacValue': '', 'NewValue': '', 'NewValueType': '', 'ObjectName': '', 'ObjectServer': '', 'ObjectType': '', 'ObjectValueName': '', 'OemInformation': '', 'OldMaxUsers': '', 'OldRemark': '', 'OldShareFlags': '', 'OldUacValue': '', 'OldValue': '', 'OldValueType': '', 'OperationType': '', 'PackageName': '', 'ParentProcessName': 'C:\\Program Files\\Microsoft Monitoring Agent\\Agent\\MonitoringHost.exe', 'PasswordHistoryLength': '', 'PasswordLastSet': '', 'PasswordProperties': '', 'PreviousDate': '', 'PreviousTime': '', 'PrimaryGroupId': '', 'PrivateKeyUsageCount': '', 'PrivilegeList': '', 'Process': 'cscript.exe', 'ProcessId': '0x12ec', 'ProcessName': '', 'Properties': '', 'ProfilePath': '', 'ProtocolSequence': '', 'ProxyPolicyName': '', 'QuarantineHelpURL': '', 'QuarantineSessionID': '', 'QuarantineSessionIdentifier': '', 'QuarantineState': '', 'QuarantineSystemHealthResult': '', 'RelativeTargetName': '', 'RemoteIpAddress': '', 'RemotePort': '', 'Requester': '', 'RequestId': '', 'RestrictedAdminMode': '', 'RowsDeleted': '', 'SamAccountName': '', 'ScriptPath': '', 'SecurityDescriptor': '', 'ServiceAccou

In [ ]:
# Type some code


<a id='moresections'></a>[Contents](#toc)
# More Sections Here

<a id='appendices'></a>[Contents](#toc)
# Appendices

## Available DataFrames

In [ ]:
print('List of current DataFrames in Notebook')
print('-' * 50)
current_vars = list(locals().keys())
for var_name in current_vars:
    if isinstance(locals()[var_name], pd.DataFrame) and not var_name.startswith('_'):
        print(var_name)

## Saving Data to CSV
To save the contents of a pandas DataFrame to an CSV
use the following syntax
```
host_logons.to_csv('host_logons.csv')
```

## Saving Data to Excel
To save the contents of a pandas DataFrame to an Excel spreadsheet
use the following syntax
```
writer = pd.ExcelWriter('myWorksheet.xlsx')
my_data_frame.to_excel(writer,'Sheet1')
writer.save()
```